In [255]:
# !pip install -U torch transformers accelerate sentence-transformers
# !pip install -U torch transformers accelerate sentence-transformers
# !pip uninstall spellchecker -y
# !pip install pyspellchecker
# !pip install unidecode
# !pip install scikit-learn  # For cosine_similarity and numpy (it's a dependency)
# !pip install sentence-transformers
# !pip install pandas
#!pip install ipywidgets

import re
import unidecode
from spellchecker import SpellChecker
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import datadetox as detox
import importlib
importlib.reload(detox)

<module 'datadetox' from 'c:\\Users\\USER\\Desktop\\GitHub\\Brasileirao-Analise\\3. Source Code\\Transformed\\datadetox.py'>

Leitura e padronização de colunas de chaves primarias para concatenação de dfs

In [312]:
pd.set_option('display.max_columns', None)
path = r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed'

#Games 
df_list = []
for i in range(1, 7):
    df = pd.read_excel(path + f'\\0{i}_games.xlsx')
    df_list.append(df)

#fitragem série B comjumto 5
df_list[4] = df_list[4][df_list[4]['Série']=='A']

# mapeamento_colunas chave primária
mapeamento_colunas = {  'data': 'Data','Date': 'Data','data_partida': 'Data',
                        'time_mandante': 'Mandante','mandante': 'Mandante','Home': 'Mandante',
                        'time_visitante': 'Visitante','visitante': 'Visitante','Away': 'Visitante'}
    

df_list = [df.rename(columns=mapeamento_colunas) for df in df_list]

df_list = detox.padronizar_datas(df_list)

DataFrame 0: Já é datetime64, mantido como está.
DataFrame 1: Convertendo padrão com '/'
DataFrame 2: Convertendo padrão com '/'
DataFrame 3: Convertendo padrão com '/'
DataFrame 4: Convertendo padrão ISO com '-'
DataFrame 5: Já é datetime64, mantido como está.


Unir todos os DFs com agrupamento por colunas, Data, Mandante e Visitante

In [313]:
#mapear cluber qeu o filtro BERT nao resolve (manualmente)
clubes_map_inicial = {  "America-RN":"América Natal","América-RN":"América Natal","América rn":"América Natal",
                        "Joinvile":"Joinville",
                        "Atlético pr":"Athlético-PR","Atlético-PR":"Athlético-PR", 
                        "Grêmio Prudente":"Barueri","Grêmio prudente":"Barueri","Gremio Prudente":"Barueri"}

for df in df_list:
    df['Mandante'] = df['Mandante'].replace(clubes_map_inicial)
    df['Visitante'] = df['Visitante'].replace(clubes_map_inicial)

# listar clubes Mandantes e visitantes, para agrupar ortograficamente
clubes_unicos = []
for df in df_list:
    unicos_por_bd = pd.unique(np.concatenate([df['Mandante'].values, df['Visitante'].values])).tolist()
    clubes_unicos = list(set(clubes_unicos) | set(unicos_por_bd))

clubes_value_dict = [   "América Mineiro", "América Natal", "Athletico Paranaense", "Atlético Goianiense", "Atlético Mineiro", "Avaí", "Bahia", 
                        "Barueri", "Botafogo", "Red Bull Bragantino", "Brasiliense", "Ceará", 
                        "Chapecoense", "Corinthians", "Coritiba", "Criciúma", "Cruzeiro", "CSA", "Cuiabá", "Figueirense", "Flamengo", "Fluminense", 
                        "Fortaleza","Guarani", "Goiás", "Grêmio", "Grêmio Prudente", "Internacional", "Ipatinga", "Joinville", "Juventude","Mirassol", "Náutico", 
                        "Palmeiras", "Paraná", "Paysandu", "Ponte Preta", "Portuguesa", "Santa Cruz", "Santo André", "Santos", "São Caetano", "São Paulo", 
                        "Sport Recife", "Vasco da Gama", "Vitória"]

modelo = SentenceTransformer('all-MiniLM-L6-v2')
clubes_corrigidos = detox.corrigir_times_semanticamente(clubes_unicos, clubes_value_dict,modelo)
clubes_dict = detox.gerar_dicionario_mapeamento(clubes_corrigidos, limite=0.0)

# corrigindo e agrupando clubes em todos os dataframes
for df in df_list:
    df['Mandante'] = df['Mandante'].replace(clubes_dict)
    df['Visitante'] = df['Visitante'].replace(clubes_dict)

# unindo todos os dfs com merge on=['Data','Mandante','Visitante']
df_merged = pd.DataFrame(columns=['Data','Mandante','Visitante'])
for i,df in enumerate(df_list):
    df_merged = pd.merge(df_merged, df, how='outer', on=['Data','Mandante','Visitante'],suffixes=('', f'_{i}'))


In [314]:
pd.set_option('display.max_rows', None)
import datetime as dt
df = df_merged.copy()

# Criar edições dos campeonatos visto que 2020 foi ate 2021
mask = (df['Data'] > dt.datetime(2020, 8, 1)) & (df['Data'] < dt.datetime(2021, 2, 28))
df.loc[mask, 'edition'] = 2020
df.loc[~mask, 'edition'] = df.loc[~mask, 'Data'].dt.year

df.sort_values(by=['Mandante', 'Visitante','edition','Data'], ascending=[True, True, True, True], inplace=True)
#df.head(10)

In [315]:
importlib.reload(detox)

# Agrupar e aplicar a função
df_ajustado = df.groupby(['Mandante', 'Visitante', 'edition'], group_keys=False).apply(detox.ajustar_datas)

# Aplica o agrupamento e mescla as linhas sem conflito
df_mesclado = (
    df_ajustado.groupby(['Data', 'Mandante', 'Visitante', 'edition'], group_keys=False)
      .apply(detox.mesclar_linhas_sem_conflito)
      .reset_index(drop=True))

df_mesclado.to_excel(r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed\Agrupamento_unico.xlsx',index=False)

C:\Users\USER\AppData\Local\Temp\ipykernel_1468\521695030.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_ajustado = df.groupby(['Mandante', 'Visitante', 'edition'], group_keys=False).apply(detox.ajustar_datas)
C:\Users\USER\AppData\Local\Temp\ipykernel_1468\521695030.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  .apply(detox.mesclar_linhas_sem_conflito)
C:\Users\USER\AppData\Local\Temp\ipykernel_1468\521695030.py:9: DeprecationWarning: D

In [ ]:
df_2006 = df_list.copy()
df_2006.pop(0)
df_2006.pop(-2)
for df in df_2006:
    df['Ano_Part'] = df['Data'].dt.year
    df = df[df['Ano_Part'] == 2010]  

for df in df_2006:
    df = df[(df['Mandante'] == 'Goiás') & (df['Visitante'] == 'Flamengo') & (df['Ano_Part'] == 2010)]
    display(df)

In [284]:
df = df_mesclado.copy()
pd.set_option('display.max_info_columns', 200)  # ou None para infinito
#df.info()

df['N_partidas_mandante'] = df.groupby(['edition', 'Mandante'])['Mandante'].transform('count')
df['N_partidas_visitante'] = df.groupby(['edition','Visitante'])['Visitante'].transform('count')
df = df[df['edition']<2024] #removendo 2024

df = df[df['edition']==2010]
# df['Mandante'].value_counts()

# df = df[df['Mandante'] == 'Goiás'] #removendo jogos onde mandante e visitante sao iguais
# display(df.sort_values(by='Visitante', ascending=True))     
                                                                
# df['edition'].value_counts()
# display(df[(df['edition']==2010) & (df['N_partidas_mandante']>19) & (df['N_partidas_visitante']>19)])

In [281]:

cont_man = df_mesclado.groupby(['edition', 'Mandante']).size().reset_index(name='qtd_m')
cont_vis = df_mesclado.groupby(['edition', 'Visitante']).size().reset_index(name='qtd_v')
#display(cont_man,cont_vis)
#display(cont_man)

mapeamento = {ano: 23 for ano in [2003, 2004]}
mapeamento.update({2005: 21})
mapeamento.update({ano: 19 for ano in range(2006, 2025)})

edt = list(map(int,df_mesclado['edition'].unique()))
for edition in edt:
    df_aux = cont_man[cont_man['edition'] == edition]
    df_aux2 = cont_vis[cont_vis['edition'] == edition]
    valor_mapeado = mapeamento.get(edition)
    for i in range(len(df_aux)):
        if df_aux.iloc[i]['qtd_m'] != valor_mapeado:
            print(f'{edition}: {df_aux.iloc[i]["Mandante"]}, {df_aux.iloc[i]["qtd_m"]}')
        # if df_aux2.iloc[i]['qtd_v'] != valor_mapeado:
        #     print(f'{df_aux2.iloc[i]["Visitante"]}, {df_aux2.iloc[i]["qtd_v"]}')



# for edition in edt:
#     df_aux = df[df['edition'] == edition]
#     for i in range(len(df_aux)):
#         if not df_aux.iloc[i]['Mandante'] == mapeamento[edition]:
#             print(df_aux.iloc['Mandante'][i], df_aux.iloc['qtd_m'][i])

2004: São Paulo, 24
2005: Cruzeiro, 22
2005: Fluminense, 22
2005: Internacional, 22
2005: Juventude, 23
2005: Paysandu, 22
2005: Ponte Preta, 22
2005: Santos, 22
2005: São Paulo, 22
2005: Vasco da Gama, 23
2007: América Natal, 20
2007: Atlético Mineiro, 21
2007: Botafogo, 20
2007: Corinthians, 20
2007: Cruzeiro, 20
2007: Figueirense, 21
2007: Flamengo, 20
2007: Fluminense, 20
2007: Internacional, 20
2007: Juventude, 21
2007: Paraná, 21
2007: Santos, 20
2007: Sport Recife, 22
2007: São Paulo, 21
2007: Vasco da Gama, 22
2008: Athletico Paranaense, 20
2008: Cruzeiro, 20
2008: Figueirense, 20
2008: Flamengo, 20
2008: Grêmio, 20
2008: Ipatinga, 20
2008: Náutico, 20
2008: Palmeiras, 20
2008: Santos, 20
2008: Sport Recife, 20
2008: São Paulo, 20
2009: Atlético Mineiro, 20
2009: Barueri, 20
2009: Botafogo, 20
2009: Coritiba, 20
2009: Fluminense, 20
2009: Goiás, 20
2009: Internacional, 20
2009: Palmeiras, 20
2010: Athletico Paranaense, 29
2010: Atlético Goianiense, 20
2010: Atlético Mineiro, 30

In [320]:
df = df_mesclado.copy()
#df = df[(df['edition'] == 2010) ]
df = df[(df['edition'] == 2010) & 
        (~df['Mandante'].isin(['Náutico', 'Sport Recife','Santo André', 'Coritiba'])) & 
        (~df['Visitante'].isin(['Náutico', 'Sport Recife','Santo André', 'Coritiba']))]
print(df['Visitante'].value_counts())
#display(df.sort_values(by=['Mandante','Visitante'], ascending=[True,True]))



Visitante
Santos                  20
Vasco da Gama           20
Athletico Paranaense    20
Fluminense              20
Cruzeiro                20
Goiás                   20
São Paulo               20
Grêmio                  19
Vitória                 19
Barueri                 19
Atlético Mineiro        19
Atlético Goianiense     19
Flamengo                19
Guarani                 19
Avaí                    19
Internacional           19
Corinthians             19
Ceará                   19
Botafogo                19
Palmeiras               19
Name: count, dtype: int64
